In [1]:
from pathlib import Path
import pandas as pd
import re

In [2]:
job_apps_dir = Path("Application/")

csv_files = list(job_apps_dir.rglob("*.csv"))

In [3]:
dfs = {}
for file in csv_files:
    # Extract the job type from the csv filename
    job_type = file.stem.split("-application")[0] 
    
    df = pd.read_csv(file)
    df["job_type"] = job_type  # add new column
    
    dfs[file.stem.split("-application")[0]] = df
job_apps_types = list(dfs.keys())
all_applications = pd.concat(dfs[x] for x in job_apps_types)

In [16]:
all_applications

,Title,Address,What is your earliest available date,Submission ID,Submission Create Date,Submission Status,Submission Serial Number,User Id,Submitter Device,Submitter IP,Submitter Browser,Source URL,Submission Admin View URL,job_type,Notes
0,Mrs,"San Jose, California, United States (US)",15/07/2025,44,2025-07-10 15:43:56,unread,4,0,Apple,98.207.87.163,Firefox,https://cadware.house/apply-backend-developer-...,https://cadware.house/wp-admin/admin.php?page=...,back-end-developer,NaN
1,Miss,"Northampton, Northampton, United Kingdom (UK)",10/07/2025,42,2025-07-09 08:59:22,unread,3,0,iPhone,82.2.149.206,iPhone,https://cadware.house/apply-backend-developer-...,https://cadware.house/wp-admin/admin.php?page=...,back-end-developer,NaN
2,Miss,"Northampton, Northampton, United Kingdom (UK)",07/07/2025,35,2025-07-02 07:00:39,unread,2,0,iPhone,82.2.149.206,iPhone,https://cadware.house/apply-backend-developer-...,https://cadware.house/wp-admin/admin.php?page=...,back-end-developer,NaN
3,Mr,"Welling, Kent, United Kingdom (UK)",01/07/2025,30,2025-06-26 18:36:25,unread,1,0,Android,31.113.246.40,Chrome,https://cadware.house/apply-backend-developer-...,https://cadware.house/wp-admin/admin.php?page=...,back-end-developer,NaN
0,Mrs,"Ado-Ekiti, Ekiti, Nigeria",01/07/2025,32,2025-06-29 13:32:16,unread,3,0,Android,197.211.63.136,Chrome,https://cadware.house/apply-content-writer-app...,https://cadware.house/wp-admin/admin.php?page=...,content-writer,NaN
1,Mr,"London, United Kingdom (UK)",28/06/2025,27,2025-06-25 11:28:10,unread,2,0,Windows,31.94.18.230,Chrome,https://cadware.house/apply-content-writer-app...,https://cadware.house/wp-admin/admin.php?page=...,content-writer,NaN
0,Miss,"exeter, devon, United Kingdom (UK)",26/07/2025,59,2025-07-18 17:35:02,unread,6,0,iPhone,85.255.236.89,iPhone,https://cadware.house/apply-data-scientist-app...,NaN,data-scientist,NaN
1,Mrs,"Gateshead, Tyne and Wear, United Kingdom (UK)",14/07/2025,41,2025-07-05 01:49:16,unread,5,0,Android,86.186.172.210,Chrome,https://cadware.house/apply-data-scientist-app...,NaN,data-scientist,NaN
2,Mr,"Gravesend, United Kingdom (UK)",22/07/2025,40,2025-07-03 22:03:48,unread,4,0,Android,109.146.147.1,Chrome,https://cadware.house/apply-data-scientist-app...,NaN,data-scientist,NaN
3,Mr,"Harrow, United Kingdom (UK)",31/07/2025,39,2025-07-03 21:30:23,unread,3,0,iPhone,82.17.78.131,Chrome,https://cadware.house/apply-data-scientist-app...,NaN,data-scientist,NaN


In [5]:
reduced_all_applications = all_applications.drop(columns = ['User Id', 'Notes', 'Submission Admin View URL', 'Submitter IP', 'Submitter Browser', 'Submitter Device', 'Submission ID', 'Submission Serial Number', 'Source URL', 'Submission Status'])

In [6]:
gender_map = {
    "Mr": "Male",
    "Mrs": "Female",
    "Miss": "Female"
}

reduced_all_applications["gender"] = reduced_all_applications["Title"].map(gender_map)

reduced_all_applications.drop('Title',axis=1, inplace=True)

In [7]:
def extract_country(address):
    if pd.isna(address):
        return None
    match = re.search(r'\((.*?)\)$', address.strip())
    if match:
        return match.group(1)
    return address.split(',')[-1].strip()

def extract_city(address):
    if pd.isna(address):
        return None
    parts = address.split(',')
    return parts[0].strip()

def extract_state(address):
    if pd.isna(address):
        return None
    parts = address.split(',')
    if len(parts) > 2:
        return parts[1].strip()
    elif len(parts) == 2:
        # Only city and country
        return None
    return None

reduced_all_applications["Country"] = reduced_all_applications["Address"].apply(extract_country)
reduced_all_applications["City"] = reduced_all_applications["Address"].apply(extract_city)
reduced_all_applications["State/Region"] = reduced_all_applications["Address"].apply(extract_state)

In [8]:
reduced_all_applications.rename(columns={"What is your earliest available date": "Earliest Available Date"}, inplace=True)
reduced_all_applications.rename(columns={"Submission Create Date": "Submission Date"}, inplace=True)

In [9]:
reduced_all_applications['Earliest Available Date'] = pd.to_datetime(reduced_all_applications['Earliest Available Date'])

reduced_all_applications["Available_DayOfWeek"] = reduced_all_applications["Earliest Available Date"].dt.day_name()
reduced_all_applications["Available_Month"] = reduced_all_applications["Earliest Available Date"].dt.month_name()
reduced_all_applications["Available_Year"] = reduced_all_applications["Earliest Available Date"].dt.year
reduced_all_applications

C:\Users\Ashish Kumar\AppData\Local\Temp\ipykernel_27268\2666396485.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  reduced_all_applications['Earliest Available Date'] = pd.to_datetime(reduced_all_applications['Earliest Available Date'])


,Address,Earliest Available Date,Submission Date,job_type,gender,Country,City,State/Region,Available_DayOfWeek,Available_Month,Available_Year
0,"San Jose, California, United States (US)",2025-07-15,2025-07-10 15:43:56,back-end-developer,Female,US,San Jose,California,Tuesday,July,2025
1,"Northampton, Northampton, United Kingdom (UK)",2025-07-10,2025-07-09 08:59:22,back-end-developer,Female,UK,Northampton,Northampton,Thursday,July,2025
2,"Northampton, Northampton, United Kingdom (UK)",2025-07-07,2025-07-02 07:00:39,back-end-developer,Female,UK,Northampton,Northampton,Monday,July,2025
3,"Welling, Kent, United Kingdom (UK)",2025-07-01,2025-06-26 18:36:25,back-end-developer,Male,UK,Welling,Kent,Tuesday,July,2025
0,"Ado-Ekiti, Ekiti, Nigeria",2025-07-01,2025-06-29 13:32:16,content-writer,Female,Nigeria,Ado-Ekiti,Ekiti,Tuesday,July,2025
1,"London, United Kingdom (UK)",2025-06-28,2025-06-25 11:28:10,content-writer,Male,UK,London,None,Saturday,June,2025
0,"exeter, devon, United Kingdom (UK)",2025-07-26,2025-07-18 17:35:02,data-scientist,Female,UK,exeter,devon,Saturday,July,2025
1,"Gateshead, Tyne and Wear, United Kingdom (UK)",2025-07-14,2025-07-05 01:49:16,data-scientist,Female,UK,Gateshead,Tyne and Wear,Monday,July,2025
2,"Gravesend, United Kingdom (UK)",2025-07-22,2025-07-03 22:03:48,data-scientist,Male,UK,Gravesend,None,Tuesday,July,2025
3,"Harrow, United Kingdom (UK)",2025-07-31,2025-07-03 21:30:23,data-scientist,Male,UK,Harrow,None,Thursday,July,2025


In [10]:
# Ensure 'Submission Date' is in datetime format
reduced_all_applications['Submission Date'] = pd.to_datetime(reduced_all_applications['Submission Date'])

# Extract year, month (as full name), day
reduced_all_applications['submit_year'] = reduced_all_applications['Submission Date'].dt.year
reduced_all_applications['submit_month'] = reduced_all_applications['Submission Date'].dt.strftime('%B')
reduced_all_applications['submit_day'] = reduced_all_applications['Submission Date'].dt.day

# Define function to categorize time of day
def categorize_time_of_day(hour):
    if 0 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    else:
        return 'Evening'

# Extract hour and apply time of day categorization
reduced_all_applications['submit_time_of_day'] = reduced_all_applications['Submission Date'].dt.hour.apply(categorize_time_of_day)

In [17]:
reduced_all_applications.columns

Index(['Address', 'Earliest Available Date', 'Submission Date', 'job_type',
       'gender', 'Country', 'City', 'State/Region', 'Available_DayOfWeek',
       'Available_Month', 'Available_Year', 'submit_year', 'submit_month',
       'submit_day', 'submit_time_of_day'],
      dtype='object')